<a target="_blank" href="https://colab.research.google.com/github/akramdhaifullah/big-data-analysis/blob/master/malware/pcap_ml_uts.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv('tcp.csv',
                 encoding='ISO-8859-1'
                )

In [3]:
df.head()

,time,src_mac,src_ip,src_port,dest_ip,dest_port,encapsulation_type,packet_type,protocol,packet_length,...,fragment_offset,tcp_flags,seq_num,window_size,ack_num,tcp_options,is_duplicate,severity,tcp_rule,info
0,2023-01-30 01:00:07,00:0c:29:52:50:2e,10.10.1.198,59620,172.217.194.95,443,Linux cooked-mode capture v1,Sent by us,TLSv1.2,64468,...,0,0x018,1,501,1,0101080a705310265824ea67,NaN,Warning,TCP,Ignored Unknown Record
1,2023-01-30 01:00:07,00:0c:29:52:50:2e,10.10.1.198,59620,172.217.194.95,443,Linux cooked-mode capture v1,Sent by us,TLSv1.2,64468,...,0,0x018,64401,501,1,0101080a705310275824ea67,NaN,Warning,TCP,Ignored Unknown Record
2,2023-01-30 01:00:07,00:0c:29:52:50:2e,10.10.1.198,59620,172.217.194.95,443,Linux cooked-mode capture v1,Sent by us,TLSv1.2,64468,...,0,0x018,128801,501,1,0101080a705310275824ea67,NaN,Warning,TCP,Ignored Unknown Record
3,2023-01-30 01:00:07,00:0c:29:52:50:2e,10.10.1.198,59620,172.217.194.95,443,Linux cooked-mode capture v1,Sent by us,TLSv1.2,64468,...,0,0x018,193201,501,1,0101080a705310275824ea67,NaN,Warning,TCP,Ignored Unknown Record
4,2023-01-30 01:00:07,00:0c:29:52:50:2e,10.10.1.198,59620,172.217.194.95,443,Linux cooked-mode capture v1,Sent by us,TLSv1.2,64468,...,0,0x018,257601,501,1,0101080a705310285824ea67,NaN,Warning,TCP,Ignored Unknown Record


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253391 entries, 0 to 253390
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   time                253391 non-null  object
 1   src_mac             253391 non-null  object
 2   src_ip              253391 non-null  object
 3   src_port            253391 non-null  int64 
 4   dest_ip             253391 non-null  object
 5   dest_port           253391 non-null  int64 
 6   encapsulation_type  253391 non-null  object
 7   packet_type         253391 non-null  object
 8   protocol            253391 non-null  object
 9   packet_length       253391 non-null  int64 
 10  dsp                 253391 non-null  object
 11  ip_flags            253391 non-null  object
 12  fragment_offset     253391 non-null  int64 
 13  tcp_flags           253391 non-null  object
 14  seq_num             253391 non-null  int64 
 15  window_size         253391 non-null  int64 
 16  ac

In [ ]:
# function to label if a frame is suspicious or not
def label_suspicious(row):
    if (row['packet_length'] > 65000) or (row['tcp_flags'] == "0x018") or (row['seq_num'] > 5000):
        return 'suspicious'
    # elif (row['src_ip'] not in trusted_ip_list) or (row['dest_ip'] not in trusted_ip_list):
    #     return 'Suspicious'
    # elif (row['packet_length'] > 20000 and row['time'] < "2023-01-30 04:00:00"):
    #     return 'Suspicious'
    # elif row['dest_port'] in suspicious_ports:
    #     return 'Suspicious'
    # elif row['encapsulation_type'] not in trusted_encapsulation_types:
    #     return 'Suspicious'
    # elif row['severity'] == "High":
    #     return 'Suspicious'
    else:
        return 'normal'

In [29]:
df.dropna(inplace=True)

In [6]:
df.drop(columns=['time', 'info'], inplace=True)

In [10]:
encoder = LabelEncoder()

df_encoded = df

for feature in df:
    if df[feature].dtype == 'O':
        df_encoded[feature] = encoder.fit_transform(df[feature])

df_encoded

,src_mac,src_ip,src_port,dest_ip,dest_port,encapsulation_type,packet_type,protocol,packet_length,dsp,ip_flags,fragment_offset,tcp_flags,seq_num,window_size,ack_num,tcp_options,is_duplicate,severity,tcp_rule
0,2,1,59620,5,443,0,0,4,64468,0,1,0,4,1,501,1,201901,1,2,2
1,2,1,59620,5,443,0,0,4,64468,0,1,0,4,64401,501,1,201902,1,2,2
2,2,1,59620,5,443,0,0,4,64468,0,1,0,4,128801,501,1,201902,1,2,2
3,2,1,59620,5,443,0,0,4,64468,0,1,0,4,193201,501,1,201902,1,2,2
4,2,1,59620,5,443,0,0,4,64468,0,1,0,4,257601,501,1,201903,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253386,2,1,59620,5,443,0,0,4,14068,0,1,0,4,474644257,742,27328,207256,1,3,2
253387,3,5,443,1,59620,0,1,3,80,1,0,0,1,27328,15371,473559297,201894,0,1,0
253388,3,5,443,1,59620,0,1,3,88,1,0,0,1,27328,15371,473559297,201895,0,1,0
253389,3,5,443,1,59620,0,1,3,96,1,0,0,1,27328,15371,473559297,201896,0,1,0


In [11]:
scaler = StandardScaler()

df_scaled = scaler.fit_transform(df_encoded)

# feature selection

In [35]:
X = df.drop(['Severity'], axis=1)
y = df['Severity']

In [36]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')  # Choose the appropriate strategy
X_imputed = imputer.fit_transform(X)

# SMOTE()

In [37]:
oversample = SMOTE()

X_sampled, y_sampled = oversample.fit_resample(X_imputed, y)

# splitting

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)

In [43]:
clf = HistGradientBoostingClassifier()

clf.fit(X_train, y_train)

HistGradientBoostingClassifier()

In [44]:
y_pred = clf.predict(X_test)

In [45]:
acc_score = accuracy_score(y_test, y_pred)
clf_report = classification_report(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

print(f"Accuracy:\n{acc_score}\n\n{clf_report}\nConfusion matrix:\n{conf_mat}")

Accuracy:
0.9915321011211422

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     35056
           1       0.98      1.00      0.99     35104
           2       1.00      0.97      0.99     35179

    accuracy                           0.99    105339
   macro avg       0.99      0.99      0.99    105339
weighted avg       0.99      0.99      0.99    105339

Confusion matrix:
[[35056     0     0]
 [    0 35104     0]
 [    0   892 34287]]
